In [69]:
import pyodbc
import pandas as pd
import numpy as np
from konlpy.tag import Okt , Kkma
import re
from tqdm import tqdm
from collections import Counter

In [3]:
# 빈도 분석
DSNNAME = "Tibero6"
DBUSER = "JH"
DBPWD = "nare"

cnxn = pyodbc.connect('DSN='+DSNNAME+';UID='+DBUSER+';PWD='+DBPWD)
cursor = cnxn.cursor()
cursor.execute("SELECT REST_ID , REVIEW , LABEL from POSNEG_V2;")

row = cursor.fetchall()

cnxn.commit()
cnxn.close()

In [10]:
row[0][0]

'1028483996'

In [13]:
lst = []
for i in row :
    lst.append(i)

In [17]:
row_frm = pd.DataFrame.from_records(lst, columns=['REST_ID' , 'REVIEW' ,'LABEL'])

In [18]:
row_frm

,REST_ID,REVIEW,LABEL
0,1028483996,김치찌개 돈까스 제육 다 맛있어요 특히 저 계란말이가 미쳤어요ㅠㅠㅠㅠ,2
1,1028483996,가끔 가는곳이여요,2
2,1028483996,집밥먹는기분이에요,1
3,1028483996,굿,2
4,1028483996,가성비 굿~ 맛있어요,0
...,...,...,...
77504,1971746359,친절해요,2
77505,1971746359,친절해요,2
77506,1971746359,굿,2
77507,1971746359,대화하기 좋아요,2


In [19]:
row_group = row_frm.groupby('REST_ID')
group_id = list(set(row_frm.REST_ID.tolist()))

In [21]:
group_list = []
for gp in group_id :
    # print(f'group_{gp}')
    globals()[f'group_{gp}'] = row_group.get_group(gp)
    group_list.append(globals()[f'group_{gp}'])

In [22]:
group_list

[          REST_ID                                             REVIEW LABEL
 77285  1526105715                                                  ㅎ     1
 77286  1526105715                                                  ㅇ     1
 77287  1526105715                                                  굿     2
 77288  1526105715                                                  굿     2
 77289  1526105715  수목원 야시장 - 랍스타 이용했어요.(근데 상호가 왜 여긴지 모르겠네요)\n양도 적...     1
 77290  1526105715                                    돈까스랑 수제어묵 맛있어요!     1
 77291  1526105715                                           그냥 뭐 먹을만     1
 77292  1526105715      뷰가 너무멋져요 음식도 맛있구여. 제주도에 이가격이면 가성비 최고 맛집 같습니다!     0
 77293  1526105715                                          돈가스 맛있어요~     2
 77294  1526105715                                  맛있어요. 친절해요. 뷰좋아요.     0
 77295  1526105715                                     제스타일은 아니었어요 ㅠㅠ     1
 77296  1526105715                                         분위기도 맛도 최고     0
 77297  1526

In [189]:
wordcloud_list = []
okt =Okt()
kkma = Kkma()
pbar = tqdm(group_list)
# 차원 분할, 어절 추출
def pos(copy_review) :
    split_review = copy_review.REVIEW.tolist()
    # 3차원 구조, 2차원으로 변환
    chage_space = [item for sublist in split_review for item in sublist ]
    
    # 명사와 형용사만 추출
    encoded = [word for word,shape in chage_space if shape in ['Adjective' , 'Noun'] if word not in stopwords]
    # 단어 묶기
    encoded = ['최고' if word == '굿' else word for word in encoded]

    # print(encoded)
    # 개수 세기
    word_counts = Counter(encoded)
    #df = pd.DataFrame.from_dict(word_counts, orient='index', columns=['COUNT'])
    df = pd.DataFrame.from_records(list(word_counts.items()), columns=['WORD', 'WORD_COUNT'])
    df = df.sort_values('WORD_COUNT' , ascending=False).reset_index(drop=True)
    # df.loc[df['WORD'] == '가성', 'WORD'] = '가성비'
    # df.loc[df['WORD'] == '그렇다', 'WORD'] = '그저그렇다'
    
    # WORD Finder 단어 
    df['WORD_FINDER'] = df.WORD.apply(lambda x : kkma.pos(x))
    
    word_finder_lst = df.WORD_FINDER.tolist()
    word_finder_inframe = []
    
    for loc in word_finder_lst :
        rm_lst = []
        for word , pos in loc :
            if pos == 'VA' or pos =='NNG' or pos =='MAG' or pos =='NNP' or pos == 'XR' or pos =='UN' or pos =='VV' :
                if word != '하다' :
                    rm_lst.append(word)
        if rm_lst == ['라','떼'] :
            rm_lst = ['라떼']
            
        word_finder_inframe.append(rm_lst)
    
    
    df['WORD_FINDER'] = word_finder_inframe

    # 단어 고치기
    df['WORD'].replace({'가성':'가성비','그렇다':'그저그렇다','엄마':'엄마음식','중국':'중국집','없다':'맛없다','많다':'양이 많다','인분':'다인분',
                       '맘':'마음'}, inplace=True)
    
    result = df[:20]
    
    return result
                          
for group_once in pbar :
    
    stopwords = [ '도', '는', '다', '의', '가', '이', '은', 
                  '한', '에', '하', '고', '을', '를', '인', '듯', 
                  '과', '와', '네', '들', '듯', '지', '임', '게', 
                  '는', '이', '했', '슴', '음', '것', '거', '로',
                  '들', '거', '곳', '분', '원', '더', '왜', '해',
                  '수', '할', '그', '함', '돈', '번', '두', '개',
                  '건', '내', '저', '만', '갈', '걸', '제', '명', 
                  '분', '또', '및', '안', '른', '때', '접', '나',
                  '막', '비', '짐', '찌', '햇', '좀', '후', '요',
                  '찰', '날', '꼭',
                  '주시', '여기', '완전', '먹기', '하나', '페스', 
                  '인데', '이가', '했었', '해서', '있다', '이다',
                  '진짜', '유학', '맞이', '같다', '그니',
                  '그냥', '이렇다', '세상', '아니다', 
                  '습니다', '했는데', '입니다' ]
    
    # 정규 표현식 수행
    copy_review = group_once.copy()
    copy_review.REVIEW = copy_review.REVIEW.str.replace(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','' , regex=True)
    copy_review.dropna(subset=['REVIEW'], inplace=True)
    
    if len(copy_review) == 0 :
        no_wordcloud = pd.DataFrame({'WORD':None , 'COUNT':None ,'LABEL_ATF':None , 'REST_ID':int(copy_review.REST_ID.iloc[0])})
        wordcloud_list.append(no_wordcloud)
    else : 
        # okt pos 어절 분할
        copy_review.REVIEW = copy_review.REVIEW.apply(lambda x: okt.pos(x, norm=True, stem=True))
        copy_review.dropna(subset=['REVIEW'], inplace=True)
    
        if len(copy_review) == 0 :
            no_wordcloud = pd.DataFrame({'WORD':None , 'COUNT':None ,'LABEL_ATF':None , 'REST_ID':int(copy_review.REST_ID.iloc[0])})
            wordcloud_list.append(no_wordcloud)
        else :
#             T_copy_review = copy_review.query('LABEL_TF == "T"')
#             F_copy_review = copy_review.query('LABEL_TF == "F"')

            word_counts = pos(copy_review) # all
#             word_counts['LABEL_ATF'] = 'A'
#             T_word_counts = pos(T_copy_review) # T
#             T_word_counts['LABEL_ATF'] = 'T'
#             F_word_counts = pos(F_copy_review) # F
#             F_word_counts['LABEL_ATF'] = 'F'

    #         print(word_counts)
    #         print(T_word_counts)
    #         print(F_word_counts)

            # once_wordcloud = pd.concat([word_counts , T_word_counts , F_word_counts] , axis = 0).reset_index(drop=True)
            once_wordcloud = word_counts
            once_wordcloud['REST_ID'] = copy_review.REST_ID.iloc[0]
            wordcloud_list.append(once_wordcloud)
            # print(once_wordcloud)
    #print('------------------------------------one group end------------------------------------')
pbar.close()
wordcloud_frm = pd.concat(wordcloud_list , axis=0)

100%|████████████████████████████████████████████████████████████████████████████████| 881/881 [07:52<00:00,  1.87it/s]


In [1]:
wordcloud_frm

NameError: name 'wordcloud_frm' is not defined

In [191]:
wordcloud_frm.to_excel('wordcloud_v2.xlsx', index=False)

In [186]:
wordcloud_frm.WORD_FINDER.loc[5]

5     [좋]
5     [맛]
5    [라떼]
5     [없]
Name: WORD_FINDER, dtype: object

In [29]:
pd.set_option('display.max_rows', None)

In [122]:
kkma.pos('맛집')

[('맛', 'NNG'), ('집', 'XSN')]

In [123]:
kkma.pos('깨끗하다')

[('깨끗하', 'VV'), ('다', 'EFN')]

In [115]:
kkma.pos('하다')

[('하다', 'NNP')]

In [71]:
lst_test = ['맛있다','돈까스']

tt = pd.DataFrame(lst_test , columns =['name'])
tt

,name
0,맛있다
1,돈까스


In [73]:
tt.name = tt.name.apply(lambda x : kkma.pos(x))
tt

,name
0,"[(맛있, VA), (다, EFN)]"
1,"[(돈까스, NNG)]"


In [77]:
tt.name = tt.name.apply(lambda x : kkma.pos(x))
tt
ttl = tt.name.tolist()
for i in ttl :
    for word , pos in i :
        if pos == 'VA' or pos =='NNG' :
            print(word , pos)

맛있 VA
돈까스 NNG


In [ ]:
copy_review.REVIEW = copy_review.REVIEW.apply(lambda x: okt.pos(x, norm=True, stem=True))

In [ ]:
encoded = [word for word,shape in lst_test if shape in ['Adjective' , 'Noun'] if word not in stopwords]

In [68]:
kkma.pos('어묵')

[('어묵', 'NNG')]

In [ ]:
from konlpy.tag

In [52]:
wordcloud_frm

,WORD,WORD_COUNT,REST_ID
0,뷰,8,1526105715
1,맛,7,1526105715
2,돈까스,7,1526105715
3,최고,6,1526105715
4,맛집,6,1526105715
5,맛있어요,6,1526105715
6,어묵,6,1526105715
7,음식,5,1526105715
8,좋아요,4,1526105715
9,좋았어요,2,1526105715


In [50]:
wordcloud_frm

,WORD,WORD_COUNT,REST_ID
0,맛있다,14,1526105715
1,좋다,9,1526105715
2,뷰,8,1526105715
3,맛,7,1526105715
4,돈까스,7,1526105715
5,최고,6,1526105715
6,어묵,6,1526105715
7,맛집,6,1526105715
8,음식,5,1526105715
9,친절하다,3,1526105715
